### README

### Importing Modules

In [1]:
import requests
import time
import re
import unicodedata

### Global Constants

In [2]:
# Instagram base url preffix
tagurl_prefix = 'https://www.instagram.com/explore/tags/'

# suffix to append to tag request url to retrieve data in JSON format
tagurl_suffix = '/?__a=1'

# suffix to end cursor when requesting posts by tag
tagurl_endcursor = '&max_id='

# a generic media post preffix (concat with media shortcode to view)
posturl_prefix = 'https://www.instagram.com/p/'

### Defining Functions

In [3]:
import unicodedata

def strip_accents(text):
    
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError):
        pass
    
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    
    return str(text)

In [4]:
import re

def text2tags(text, striptag=True):
    
    pattern = '#\S+'
    
    text = text.lower()
    
    text = strip_accents(text)
    
    matches = re.findall(pattern, text)
    
    if striptag :
        matches = [ match.replace('#','') for match in matches ]
    
    return matches

In [5]:
def json2posts(json_info, infilter=False):

    posts_list = json_info['graphql']['hashtag']['edge_hashtag_to_media']['edges']

    posts_dicts = []

    for post in posts_list:

        node = post['node']

        id_post = node['id']

        id_owner = node['owner']['id']

        shortcode = node['shortcode']

        edges = node['edge_media_to_caption']['edges']
        
        text = edges[0]['node']['text'].replace('\n','') if len(edges) else ''
        
        tags = text2tags(text)

        post_url = posturl_prefix + shortcode + '/'

        post_dict = {
            'id_post': id_post,
            'id_owner': id_owner,
            'shortcode': shortcode,
            'text': text,
            'post_url': post_url,
            'tags': tags
        }
        
        if infilter :
            if len(tags) :
                posts_dicts.append( post_dict )
        else:
            pass
    
    else:
        posts_dicts.append( post_dict )
    
    return posts_dicts

In [13]:
import requests
import time

def snowball(url, deep=1, end_cursor='', count=0, showurl=False, 
             sleep=0, forever=False, progress=False, pause=60 ):

    request_url = url + tagurl_endcursor + end_cursor

    if showurl :
        print(request_url)
    else:
        if progress :
            print( count, end=' ' )
    
    while True :
        try :
            json_info = requests.get( request_url ).json()
            break
        except:
            if forever :
                print('Fail, retrying in ' + str(pause) + ' seconds')
                time.sleep(pause)
            else:
                print('Fail, ' + str(count) + ' requests done')
                return []
    
    end_cursor = json_info['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']

    posts = json2posts( json_info, True )

    time.sleep(sleep)
  
    count = count + 1

    if count < deep :
        posts += snowball(
            url=url, 
            deep=deep, 
            end_cursor=end_cursor, 
            count=count, 
            showurl=showurl, 
            sleep=sleep,
            forever=forever,
            progress=progress, 
            pause=pause)
    else:
        pass
    
    if showurl :
        pass
    else:
        if progress :
            if count == deep :
                print()

    return posts

### Collecting Data

In [7]:
# target initial tags
tags = ['bolsonaro', 'haddad', 'dilma', 'ciro', 'guedes', 'moro', 'lula']

In [8]:
# urls to initial tags
queries = [ tagurl_prefix + tag + tagurl_suffix for tag in tags ]

In [14]:
%%time

for tag, query in zip( tags, queries ) :
    
    print( 'Querying ' + tag + '...' )
    
    posts = snowball(query, deep=40, forever=True, sleep=1, pause=60, progress=True)
    
    data[tag] = posts
    
    print( 'Done' )
    
    time.sleep(30)

Querying bolsonaro...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
Done
Querying haddad...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
Done
Querying dilma...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
Done
Querying ciro...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
Done
Querying guedes...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
Done
Querying moro...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
Done
Querying lula...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
Done
CPU times: user 27.2 s, sys: 1.67 s, total: 28.9 s
Wall tim

In [17]:
# checking number of medias
for key, posts in data.items() :
    
    print(key, len(posts))

bolsonaro 2401
haddad 2627
lula 2423
moro 2548
guedes 2755
ciro 2746
dilma 2048


In [18]:
import json

# saving data to a JSON file
f = open('data.json', 'w')
json.dump(data, f)
f.close()

### Filtering Data

This code is a plus. The function `json2posts` has the boolean attribute `infilter`, which filters and drops out the posts with no tags.

In case this attribute is `False` (not recommended), we can check the non-tagged posts here. 

In [21]:
# checking non-tagged medias

for key, posts in data.items():
    
    for post in posts:
        
        tags = post['tags']
        
        if len(tags) :
            pass
        else:
            print(post['post_url'], media['text'])

https://www.instagram.com/p/BzI1Gq6BlHU/ Tempo é ouro!
https://www.instagram.com/p/BzJuI6_gtYF/ Tempo é ouro!
https://www.instagram.com/p/BgzrQ9MBZgm/ Tempo é ouro!
https://www.instagram.com/p/Bico-WAgppM/ Tempo é ouro!
https://www.instagram.com/p/BmvmyIOAUC-/ Tempo é ouro!
https://www.instagram.com/p/Bv9gTx8AKjE/ Tempo é ouro!
https://www.instagram.com/p/ByQ8OazAxwm/ Tempo é ouro!
https://www.instagram.com/p/BzI6H4fHMtt/ Tempo é ouro!
https://www.instagram.com/p/Bxj9mBngNqm/ Tempo é ouro!
https://www.instagram.com/p/BzHpyzalTvh/ Tempo é ouro!
https://www.instagram.com/p/BzG_MK9H9Er/ Tempo é ouro!
https://www.instagram.com/p/BzHFMPrgJl4/ Tempo é ouro!
https://www.instagram.com/p/Bxzv_9nlARH/ Tempo é ouro!
https://www.instagram.com/p/BzEJgO3Hlh5/ Tempo é ouro!
https://www.instagram.com/p/By0csU7nXZo/ Tempo é ouro!
https://www.instagram.com/p/BzJO0_XF7yB/ Tempo é ouro!
https://www.instagram.com/p/BzFzG6aD27n/ Tempo é ouro!
https://www.instagram.com/p/By8CacaFwtU/ Tempo é ouro!
https://ww

### Development

This section is for development process and can be disconsidere

In [0]:
import requests
import time

# from tqdm import tqdm, tnrange
from tqdm import tqdm_notebook as tqdm

def snowball(url, deep=1, end_cursor='', count=0, showurl=False, 
             sleep=0, forever=False, progress=False, pause=60, pbar=None ):
    
    if pbar is None :
        pbar = tqdm(total=deep)

    request_url = url + tagurl_endcursor + end_cursor

    if showurl :
        print(request_url)
    else:
        if progress :
            pbar.update()
    
    while True :
        try :
            json_info = requests.get( request_url ).json()
            break
        except:
            if forever :
                print('Fail, retrying in ' + str(pause) + ' seconds...', end=' ')
                time.sleep(pause)
                print('Go!')
            else:
                print('Fail, ' + str(count) + ' requests done')
                return []
    
    end_cursor = json_info['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']

    medias = json2medias( json_info, True )

    time.sleep(sleep)
  
    count = count + 1

    if count < deep :
        medias += snowball(
            url=url, 
            deep=deep, 
            end_cursor=end_cursor, 
            count=count, 
            showurl=showurl, 
            sleep=sleep,
            forever=forever,
            progress=progress, 
            pause=pause,
            pbar=pbar)
    else:
        pass
    
    if showurl :
        pass
    else:
        if progress :
            if count == deep :
                pass
                print()

    return medias

In [78]:
snowball(queries[1], deep=5, progress=True, forever=True)[0]

Fail, retrying in 60 seconds... Go!
Fail, retrying in 60 seconds... Go!



{'id_media': '2069048540710857446',
 'id_owner': '431798494',
 'mediaurl': 'https://www.instagram.com/p/By2vK98HGrm/',
 'shortcode': 'By2vK98HGrm',
 'tags': ['morotraidordapatria',
  'infuxwetrust',
  'libertemlula',
  'morocriminoso',
  'moronacadeia',
  'dallagnolnacadeia',
  'lulalivreja',
  'lulanobeldapaz2019',
  'lula',
  'lulalivre',
  'lulalibre',
  'lulavalealuta',
  'lulapresidente',
  'freelula',
  'lulainocente',
  'eusoulula',
  'lula2018',
  'lulapresopolitico',
  'partidodostrabalhadores',
  'haddad',
  'haddad13',
  'haddadpresidente',
  'obrasilfelizdenovo',
  'mulherescontrabolsonaro',
  'elenao',
  'elenao',
  'elenunca',
  'elejamais'],
 'text': 'VÍDEO PARA QUEM AINDA NÃO ENTENDEU A ATUAÇÃO DO MORO NO PROCESSO DO LULABANDIDO BOM É BANDIDO MORO!!! #morotraidordapatria #infuxwetrust #libertemlula #morocriminoso #moronacadeia #dallagnolnacadeia #lulalivreja #lulanobeldapaz2019 #lula #lulalivre #lulalibre #lulavalealuta #lulapresidente #freelula #lulainocente #eusoulula